# check gpu resources

In [ ]:
# gpu resources
! nvidia-smi

# running ManTraNet on OpenMFC dataset

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import cv2
import requests
import sys
import image_slicer
import time
from tqdm.notebook import tqdm

from PIL import Image
from io import BytesIO
from matplotlib import pyplot

In [ ]:
# ManTraNet paths
manTraNet_root = './'
manTraNet_srcDir = os.path.join( manTraNet_root, 'src' )
sys.path.insert( 0, manTraNet_srcDir )
manTraNet_modelDir = os.path.join( manTraNet_root, 'pretrained_weights' )

# setup dataset paths

In [ ]:
# set the part #
dataset_part_num = 1

In [ ]:
# OpenMFC dataset
mfc_data = os.path.join(manTraNet_root, 'openmfc_data')
openmfc_2020 = os.path.join(mfc_data, 'OpenMFC2020')

# p1 for index
openmfc_2020_p1 = os.path.join(openmfc_2020, 'OpenMFC20_Image_Ver1-part001of27')
openmfc_2020_p1_indexes = os.path.join(openmfc_2020_p1, 'indexes')

# dynamic part #
openmfc_2020_part = os.path.join(openmfc_2020, f'OpenMFC20_Image_Ver1-part00{dataset_part_num}of27')
openmfc_2020_part_probe = os.path.join(openmfc_2020_part, 'probe')
openmfc_2020_part_mask_out = os.path.join(openmfc_2020_part, 'predicted_masks')

In [ ]:
# OpenMFC dataset
mfc_data = os.path.join(manTraNet_root, 'openmfc_data')
openmfc_2020 = os.path.join(mfc_data, 'OpenMFC2020')

openmfc_2020_p1 = os.path.join(openmfc_2020, 'OpenMFC20_Image_Ver1-part001of27')
openmfc_2020_p1_indexes = os.path.join(openmfc_2020_p1, 'indexes')
openmfc_2020_p1_probe = os.path.join(openmfc_2020_p1, 'probe')
openmfc_2020_p1_mask_out = os.path.join(openmfc_2020_p1, 'predicted_masks')

# load dataset description

In [ ]:
imdl_index_path = os.path.join(openmfc_2020_p1_indexes, 'OpenMFC20_Image-IMDL-index.csv')

In [ ]:
df = pd.read_csv(imdl_index_path, sep='|')

In [ ]:
df.head()

In [ ]:
df['TaskID'].unique()

# build dataset list

In [ ]:
dataset = os.listdir(openmfc_2020_p1_probe)

# Load A Pretrained ManTraNet Model

In [ ]:
import modelCore
# 4 is default model
manTraNet = modelCore.load_pretrain_model_by_index( 4, manTraNet_modelDir )

In [ ]:
# ManTraNet Architecture 
print(manTraNet.summary(line_length=120))

In [ ]:
# Image Manipulation Classification Network
IMCFeatex = manTraNet.get_layer('Featex')
print(IMCFeatex.summary(line_length=120))

# test samples from OpenMFC

In [ ]:
from datetime import datetime 
def read_rgb_image( image_file ) :
    rgb = cv2.imread( image_file, 1 )[...,::-1]
    return rgb
    
def decode_an_image_array( rgb, manTraNet ) :
    x = np.expand_dims( rgb.astype('float32')/255.*2-1, axis=0 )
    t0 = datetime.now()
    y = manTraNet.predict(x)[0,...,0]
    t1 = datetime.now()
    return y, t1-t0

def decode_an_image_file( image_file, manTraNet ) :
    rgb = read_rgb_image( image_file )
    mask, ptime = decode_an_image_array( rgb, manTraNet )
    return rgb, mask, ptime.total_seconds()

In [ ]:
def slice_and_decode(filename, manTraNet):
    tiles = image_slicer.slice(filename, number_tiles=6)
    mask_tiles = []
    total_ptime = 0
    for tile in tiles:
        rgb, mask, ptime = decode_an_image_file(tile.filename, manTraNet)
        mask_image = Image.fromarray(np.uint8(mask * 255))
        mask_tile = image_slicer.Tile(image=mask_image, number=tile.number, position=tile.position, coords=tile.coords)
        mask_tiles.append(mask_tile)
        total_ptime += ptime
    mask_tiles = tuple(mask_tiles)
    res = image_slicer.join(mask_tiles)
    return res, total_ptime

In [ ]:
from PIL import UnidentifiedImageError

failures = []

In [ ]:
# modified for OpenMFC
start = time.time()
for sample in tqdm(dataset):
    try:
        sample_probe = openmfc_2020_p1_probe + '/' + sample

        # manipulation detection using slicing
        mask, ptime = slice_and_decode( sample_probe, manTraNet )

        # show results
        pyplot.figure( figsize=(25,25) )

        pyplot.subplot(2, 2, 1)
        sample_image = pyplot.imread(sample_probe)
        pyplot.imshow( sample_image )
        pyplot.title('Forged Image (ManTra-Net Input)')

        pyplot.subplot(2, 2, 2)
        pyplot.imshow( mask, cmap='gray' )
        pyplot.title('Predicted Mask (ManTra-Net Output w/ Slicing)')

        mask_filename = openmfc_2020_p1_mask_out + '/'+ os.path.splitext(sample)[0] + '_mask' + '.png'

        pyplot.suptitle('Decoded {} of size {} for {:.2f} seconds'.format( os.path.basename( sample_probe ), 'rgb.shape', ptime ) )

        pyplot.show()

        # save mask to mask path
        mask.save(fp=mask_filename, format='PNG')
    except UnidentifiedImageError as err:
        failures.append((sample, err))
    except tf.errors.ResourceExhaustedError as err:
        failures.append((sample, err))
    except:
        failures.append((sample, sys.exc_info()[0]))

print(f'all probes completed in {start - time.time()} seconds')